# TRI: Descriptive Analysis

Greg Lee
<br> 10.01.2020

**WHAT IS THE TOXIC RELEASE INVENTORY (TRI)** 

TRI tracks the management of certain toxic chemicals that may pose a threat to human health and the environment. U.S. facilities in different industry sectors must report annually how much of each chemical is released to the environment and/or managed through recycling, energy recovery and treatment. (A "release" of a chemical means that it is emitted to the air or water, or placed in some type of land disposal.) [1]

---

**WHICH CHEMICALS AND COMPANIES ARE REQUIRED TO REPORT?**
- Have 10 or more full-time employee equivalents
- Facility is included in the following NAICS codes:
    - Manufacturing of: food, beverage, apparel, leather, wood, paper, petroleum, coal, chemical, plastics, rubber, minerals, metals, machinery, computers, electronics, applicance, transportation, furniture and more. 
- Facility uses chemical in quanitities greater than establihed threshold in the course of a calendar year. (changes year to year)

---

**WHICH CHEMICALS ARE TRACKED?**

Those associated with:<br>

1. Cancer or other chronic human health effects
2. Significant adverse acute human health effects
3. Significant adverse environmental effects

There are currently 755 individually listed chemicals and 33 chemical categories covered by the TRI Program. Facilities that manufacture, process or otherwise use these chemicals in amounts above established levels must submit annual reporting forms for each chemical. Note that the TRI chemical list doesn't include all toxic chemicals used in the U.S.

--- 

**WHEN WAS THE PROGRAM STARTED**

In 1986, Congress passed the Emergency Planning and Community Right-to-Know Act (EPCRA) to support and promote emergency planning and to provide the public with information about releases of toxic chemicals in their community. Section 313 of EPCRA established the Toxics Release Inventory. Data is collected on a yearly basis. 

[1] - https://www.epa.gov/toxics-release-inventory-tri-program/what-toxics-release-inventory
<br>[2] - https://www.epa.gov/toxics-release-inventory-tri-program/tri-covered-industry-sectors 

In [2]:
#Libraries

#Basic py: 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import os
import math

#Geo
import geopandas as gpd
import fiona
from shapely.geometry import Point
import descartes
import contextily as ctx #Basemaps 

## Load the Data

In [3]:
#Import all data and merge into one dataframe
path = "/home/boogie2/Hanson_Lab/TRI_STILT/data/raw/Toxic_Release_Inventory_raw" 
for index, files in enumerate(os.listdir(path)):
    temp_df = pd.read_csv(path + "/" + files)

    #Columns are standard, merge the dataset on that
    if index == 0:
        tri_df = temp_df
        print('check')
    else:
        tri_df = tri_df.append(temp_df)
        
#Clean up the data
tri_df.rename(columns=lambda x: ' '.join(x.split('.')[1:]), inplace=True)
tri_df.rename(columns=lambda x: x.replace(' ',''), inplace=True)

#Select the data from 1990-1999 which is air pollution
select_tri = tri_df.loc[(tri_df['YEAR']>=1990) & (tri_df["YEAR"] < 2000)]

select_tri_air = select_tri[(select_tri['51-FUGITIVEAIR']>0) | (select_tri['52-STACKAIR']>0)]
print("From 1990-1999 there are {0} data entries and {1} features".format(select_tri.shape[0],select_tri.shape[1]))

check
From 1990-1999 there are 5733 data entries and 114 features


The loaded shape of the data is what we expect. The 114 features represent details about the location, industry and toxin. For more information on this, please visit the EPAs website (https://www.epa.gov/toxics-release-inventory-tri-program/descriptions-tri-data-terms#popup_1)

## Data Cleaning

In [4]:
#Percentage of data missing values
pd.set_option('display.max_rows', 500)
print(select_tri_air.isna().sum().sort_values()/select_tri_air.shape[0])

YEAR                       0.000000
62-M26                     0.000000
62-M24                     0.000000
62-M20                     0.000000
OFF-SITERELEASETOTAL       0.000000
62-M99                     0.000000
62-M94                     0.000000
62-M90                     0.000000
62-M79                     0.000000
62-M73                     0.000000
62-M65                     0.000000
62-M64                     0.000000
62-M67                     0.000000
62-M66                     0.000000
62-M63                     0.000000
62-M72                     0.000000
62-M82                     0.000000
62-M81                     0.000000
62-M71                     0.000000
62-M61METAL                0.000000
62-M40METAL                0.000000
62-M62                     0.000000
62-M41                     0.000000
62-M10                     0.000000
62-M28                     0.000000
62-M93                     0.000000
OFF-SITERECYCLEDTOTAL      0.000000
62-M56                     0

Examination of the missing values showcases the highest number of missing values are found in the NAIC and SIC codes. This makes sense, as not all companies had multiple codes. Additionally, approximately 23% of the data is missing parent company data and binary release data. 

We will only keep data for which there is less than 20% of missing data. 

In [5]:
clean_index = select_tri_air.isna().sum()<math.floor(select_tri_air.shape[0]*0.20)
select_tri_air_clean = select_tri_air.loc[:,clean_index.index[clean_index==True]]

#Let's collect and analyze a subset of the data which focuses on TRI air releases
select_tri_air_clean = select_tri_air_clean[['YEAR','TRIFD','FRSID','FACILITYNAME','CITY','COUNTY','ST','ZIP','LATITUDE','LONGITUDE',
                           'INDUSTRYSECTORCODE','INDUSTRYSECTOR','CHEMICAL','CAS#/COMPOUNDID','METAL','CARCINOGEN' ,
                           'UNITOFMEASURE','51-FUGITIVEAIR','52-STACKAIR','PRIMARYSIC','INDUSTRYSECTORCODE',
                                             '88-ONE-TIMERELEASE','PRODUCTIONWSTE(81-87)']]

## Selecting IARC Labelled "Known" Carcinogens

The international agency for research on cancer created a listing of carcinogens with varying confidence. While the TRI data contains a carcinogen column, we decided to use the IARC classifications for chemical filtering. We find this measure to be more conservative. 

Ratings: 
<br>1 - Known Carcinogen
<br>2A - Probably carcinogenic to humans
<br>2B - Possibly carcinogenic to humans
<br>3 - not classifiable as to its carcinogenicity to humans

In [6]:
#Import the IARC Data
iarc_df = pd.read_csv('/home/boogie2/Hanson_Lab/TRI_STILT/data/raw/IARC_Class_Full_List.csv')

#Standardize the Column Names
iarc_df.rename(columns=lambda x: x.replace(' ','_'), inplace=True)
iarc_df.rename(columns=lambda x: x.replace('.',''), inplace=True)

#The CAS numbers are not standardized between groups. Let's change that!
#A CASRN contains a first number (2-7 digits) - second number (2 digits) - third number - 1 digit=
iarc_df.CAS_No = iarc_df.CAS_No.str.replace('-','').apply('{:0>9}'.format)
iarc_df.CAS_No = iarc_df.CAS_No.str.replace('000000nan',str(np.nan))

print("There are {0} chemicals with {1} features".format(iarc_df.shape[0],iarc_df.shape[1]))

There are 1079 chemicals with 6 features


In [7]:
#Select those chemicals with a 1,2A or 2B designation (KNOWN CARCINOGENS)
known_carc = iarc_df[iarc_df.Group != "3"]
#Merge the data
carc_sel_tri_air = known_carc.merge(select_tri_air_clean, 
                                    left_on='CAS_No', 
                                    right_on='CAS#/COMPOUNDID',
                                    how='inner')

print("Data are present from {0} to {1}".format(carc_sel_tri_air['YEAR'].min(),carc_sel_tri_air['YEAR'].max()))
print("There are {0} total observations in the data".format(carc_sel_tri_air.shape[0]))

Data are present from 1990 to 1999
There are 900 total observations in the data


In [8]:
#Fix those entries which have a "see xx (other chemical)"
carc_sel_tri_air.loc[carc_sel_tri_air.Agent== "Strong-inorganic-acid mists containing sulfuric acid (see Acid mists)",
       'Group'] = '1'

carc_sel_tri_air.loc[carc_sel_tri_air.Agent== "Bis(2-ethylhexyl) phthalate (see Di(2-ethylhexyl) phthalate)",
       'Group'] = '2B'

#Clean up unnessessary/unusable information
carc_sel_tri_air = carc_sel_tri_air.drop(columns=['CAS_No','Agent','Volume', 'Year', 'Additional_information'])

In [9]:
#Histogram: total number of chemicals per each IARC bin 
fig,ax = plt.subplots(1,1,figsize=(12,6))
ax = carc_sel_tri_air['Group'].value_counts().plot.bar()
ax.set_xlabel('IARC Classification')
ax.set_ylabel('Observations')
ax.set_title('Histogram of TRI Carcinogenic Status')

plt.close()

## Descriptive Analysis

In [10]:
#Unique locations and chemicals
uniq_locs = carc_sel_tri_air.groupby(['LATITUDE', 'LONGITUDE']).size().reset_index(name='Freq')
uniq_chems = carc_sel_tri_air.groupby(['CAS#/COMPOUNDID']).size().reset_index(name='Freq')
print("There are a total of {0} unique locations".format(uniq_locs.shape[0]))
print("There are a total of {0} unique chemicals".format(uniq_chems.shape[0]))

There are a total of 113 unique locations
There are a total of 32 unique chemicals


In [11]:
#What are the most and least common chemicals
agg_chems =carc_sel_tri_air['CHEMICAL'].value_counts()
print("Most common chemical")
print(agg_chems.head(5))

print("\nLeast common chemicals")
print(agg_chems.tail(5))

Most common chemical
SULFURIC ACID (1994 AND AFTER ACID AEROSOLS&quot; ONLY)&quot;    137
NICKEL                                                  97
STYRENE                                                 84
BENZENE                                                 82
NAPHTHALENE                                             73
Name: CHEMICAL, dtype: int64

Least common chemicals
EPICHLOROHYDRIN            2
DIETHANOLAMINE             2
2,4-D                      2
4,4&#39;-METHYLENEDIANILINE    1
N,N-DIMETHYLFORMAMIDE      1
Name: CHEMICAL, dtype: int64


In [12]:
val = carc_sel_tri_air[carc_sel_tri_air['51-FUGITIVEAIR']>0].shape[0]/carc_sel_tri_air.shape[0]
val2 = carc_sel_tri_air[carc_sel_tri_air['52-STACKAIR']>0].shape[0]/carc_sel_tri_air.shape[0]
val3 = carc_sel_tri_air[(carc_sel_tri_air['51-FUGITIVEAIR']>0) & (carc_sel_tri_air['52-STACKAIR']>0)].shape[0]/carc_sel_tri_air.shape[0]

print('Percentage of TRI Releases which have only Fugitive Release: {0:.2f}%'.format(val-val3))
print('Percentage of TRI Releases which have only Stack Release: {0:.2f}%'.format(val2-val3))
print('Percentage of TRI Releases with both Fugitive and Stack: {0:.2f}%'.format(val3))

Percentage of TRI Releases which have only Fugitive Release: 0.29%
Percentage of TRI Releases which have only Stack Release: 0.18%
Percentage of TRI Releases with both Fugitive and Stack: 0.53%


In [13]:
val4 = carc_sel_tri_air[carc_sel_tri_air['88-ONE-TIMERELEASE']>0].shape[0]/carc_sel_tri_air.shape[0]
print('Percentage of TRI Releases which contain a one time release: {0:.2f}%'.format(val4*100.00))
print('Approximately 20% of the TRI values are missing this label though and remain unknown')

Percentage of TRI Releases which contain a one time release: 7.56%
Approximately 20% of the TRI values are missing this label though and remain unknown


In [14]:
#Plotting the total releases per year
fig,ax = plt.subplots(1,1,figsize=(12, 7))
carc_sel_tri_air['YEAR'].value_counts().sort_index().plot.bar(ax=ax)
ax.set_xticklabels(sorted(carc_sel_tri_air['YEAR'].value_counts().index), rotation = 45, ha="right")
ax.set_xlabel('Year')
ax.set_ylabel('Observations')
ax.set_title('TRI Occurences per Year')


plt.savefig('/home/boogie2/Hanson_Lab/TRI_STILT//figures/descriptive_analysis/Average_TRI.png')
plt.close()

## Visualizing Spatial Distributions 

In [21]:
#Where are the TRI Releases?
gdf = gpd.GeoDataFrame(carc_sel_tri_air, geometry=gpd.points_from_xy(carc_sel_tri_air.LONGITUDE, carc_sel_tri_air.LATITUDE),crs='epsg:4326')
gdf = gdf.to_crs(3857)

fig, ax = plt.subplots(figsize = (10,10)) 
gdf.plot(ax = ax,markersize=20, marker='^', color='purple')

#Add Title
plt.title("Reporting TRI Facilities from 1990-1999")

#Add Legend
plt.legend(['TRI Reporting Facility Locations'], loc='lower right')

#Add Basemap
ctx.add_basemap(ax)

#Clean Up the Plot
ax.set_axis_off()
fig.patch.set_facecolor('xkcd:white')

#Save the figure
plt.savefig('/home/boogie2/Hanson_Lab/TRI_STILT//figures/descriptive_analysis/TRI_Locations.png')
plt.close()

In [17]:
#Load the County Shapefiles
counties =fiona.open('/home/boogie2/Hanson_Lab/TRI_STILT/data/raw/utah_counties.gdb')
counties_gdf = gpd.GeoDataFrame.from_features([feature for feature in counties], crs=counties.crs)
columns = list(counties.meta["schema"]["properties"]) + ["geometry"]
counties_gdf = counties_gdf[columns]
counties_gdf = counties_gdf.to_crs(epsg=3857) 

#Get the total number per country
temp=carc_sel_tri_air
temp['sumer'] = 1
temp = temp.groupby(['COUNTY']).agg({'sumer': 'sum'}).reset_index()

#Merge on County Name
merge = temp.merge(counties_gdf, 
                   left_on='COUNTY',
                   right_on='NAME',
                   how='inner')

#Place back into a geodataframe
merge = gpd.GeoDataFrame(merge)


#Check the shape match. The rows should not change
print(temp.shape[0] == merge.shape[0])

True


In [20]:
#Which Counties have the most observations? 
fig,ax = plt.subplots(1,2,figsize=(16, 8))
carc_sel_tri_air['COUNTY'].value_counts().plot.bar(ax=ax[0])
ax[0].set_xticklabels(carc_sel_tri_air['COUNTY'].value_counts().index, rotation = 45, ha="right")
ax[0].set_xlabel('County')
ax[0].set_ylabel('Observations')
ax[0].set_title('TRI Occurences by County')

#Geographic Plotting
merge.plot(cmap='YlOrRd',
           ax=ax[1],
           column='sumer',
           legend=True,
           alpha=0.8,
           legend_kwds={'label': "Occurences within County Limits"})
ax[1].set_title("Utah TRI Occurence Locations")

#Pulling off the axis
plt.rcParams.update({'font.size': 12})
plt.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,# ticks along the top edge are off
    left=False,
    labelbottom=False,
    labelleft=False)

ctx.add_basemap(ax[1])

plt.savefig('/home/boogie2/Hanson_Lab/TRI_STILT//figures/descriptive_analysis/TRI_county_occurences.png')
plt.close()

In [22]:
#Per year, how do the pounds of released material change? 

fig,ax = plt.subplots(1,2,figsize=(16, 8))

data = []
for year in sorted(carc_sel_tri_air['YEAR'].drop_duplicates()):
    d=carc_sel_tri_air[carc_sel_tri_air['YEAR']==year]['51-FUGITIVEAIR']
    data.append(d)
    
ax[0].boxplot(data)
ax[0].set_xticklabels(sorted(carc_sel_tri_air['YEAR'].drop_duplicates()), rotation = 45, ha="right")
ax[0].set_xlabel('Year')
ax[0].set_ylabel('Lbs of Released Chemicals')
ax[0].set_title('Yearly Fugitive Releases')

data = []
for year in sorted(carc_sel_tri_air['YEAR'].drop_duplicates()):
    d=carc_sel_tri_air[carc_sel_tri_air['YEAR']==year]['52-STACKAIR']
    data.append(d)
    
ax[1].boxplot(data)
ax[1].set_xticklabels(sorted(carc_sel_tri_air['YEAR'].drop_duplicates()), rotation = 45, ha="right")
ax[1].set_xlabel('County')
ax[1].set_ylabel('Lbs of Released Chemicals')
ax[1].set_title('Yearly Stack Releases')

plt.savefig('/home/boogie2/Hanson_Lab/TRI_STILT//figures/descriptive_analysis/Stack_Fug_boxplot_yearly_releases.png')
plt.close()

In [23]:
#Per county are the releases centered in any sort of manner?
fig,ax = plt.subplots(1,2,figsize=(16, 8))

data = []
for year in sorted(carc_sel_tri_air['COUNTY'].drop_duplicates()):
    d=carc_sel_tri_air[carc_sel_tri_air['COUNTY']==year]['51-FUGITIVEAIR']
    data.append(d)
    
ax[0].boxplot(data)
ax[0].set_xticklabels(sorted(carc_sel_tri_air['COUNTY'].drop_duplicates()), rotation = 45, ha="right")
ax[0].set_xlabel('County')
ax[0].set_ylabel('Lbs of Released Chemicals')
ax[0].set_title('County Fugitive Releases')

data = []
for year in sorted(carc_sel_tri_air['COUNTY'].drop_duplicates()):
    d=carc_sel_tri_air[carc_sel_tri_air['COUNTY']==year]['52-STACKAIR']
    data.append(d)
    
ax[1].boxplot(data)
ax[1].set_xticklabels(sorted(carc_sel_tri_air['COUNTY'].drop_duplicates()), rotation = 45, ha="right")
ax[1].set_xlabel('County')
ax[1].set_ylabel('Lbs of Released Chemicals')
ax[1].set_title('County Stack Releases')


plt.savefig('/home/boogie2/Hanson_Lab/TRI_STILT//figures/descriptive_analysis/Stack_Fug_boxplot_county_releases.png')
plt.close()
